# Main jupyter. 
## This note contains:
- functions
- final dataset
- folium with my final location
    - folium is centered in the prefered location (taking into account my weight function to find it)
- the distances are found using the haversine equation (to get the closest distance, considering the curvature of Earth)

In [1]:
import folium
import pandas as pd
from haversine import haversine, Unit

# My stored variables from another jupyters

In [2]:
%store -r
%store

Stored variables and their in-db values:
facilities                    -> ['airport', 'airport', 'airport', 'airport', 'airp
kindergarten_ubi              -> [(47.771722, -122.323327), (47.77412421775164, -12
kindergarten_ubi1             -> [(-122.323327, 47.771722), (-122.08008087473401, 4
kindergarten_ubi2             -> (-73.590806, 45.488338)
kindergarten_ubi3             -> [(-73.98815083551503, 40.74512371133894), (-74.005
kindergarten_ubi4             -> [(-122.65663932116907, 45.516230500599896), (-122.
kindergarten_ubi5             -> [(-122.42151999253659, 37.773589712511665), (-121.
lat_airport                   -> ['-6.081689834590001', -5.20707988739, -5.82678985
lon_airport                   -> ['145.391998291', 145.789001465, 144.2960052490234
office_lat                    -> [34.090368, 33.022176, 40.7464969, 53.344104, 34.0
office_long                   -> [-118.393064, -117.081406, -74.0094471, -6.267494,
office_name                   -> ['Geni', 'Slacker', 'Joost', 

-----------
# FUNCTIONS

# - Measuring proximity

## closest_and_ubi function using haversine formula
- takes 2 lists (a,b) of geospatial coordinates (as long as required both, lengh-independent)
- the output is:
    - for all elements on a, the closest point to all elements of b
    - the coordinates of the closest point of b
- how it works_
    name_list_for_distances, name_list_for_coordinates = closest_and_ubi (list1, list2)
    
it is not allowed to storage functions in one jupyter-notebook and call them from another. You get a RaiseError output. So I define my functions below

In [3]:
def closest_and_ubi(offices_ubi,airports_ubi):
    '''    Esta función necesita correrse como a,b=closest_and_ubi(elem1,elem2)'''
    ubi_to_office=[]
    closest_airport = []
    for o in offices_ubi:
        distancia=[]
        airports=[]
        for a in airports_ubi:
            #if haversine(o,a)<200:
            distancia.append(haversine(o,a))
            airports.append(a)
            
        m=min(distancia)
        air=airports[distancia.index(m)]
        ubi_to_office.append(round(m,1))  #the result is given gith uncertainty of +- 0.1km
        closest_airport.append(air)

    return ubi_to_office, closest_airport

# - Creating coord tuples from lists of coordinates
- I got lists of longitudes and latitudes for all places i'm working with in different jupyter-notebooks. I'll turn it into tuples in order to use the haversine library

In [4]:
def coord_tuple(list_lat,list_long):    
    '''- input: 2 lists. list1: latitudes, list2:longitudes
    - output: list of tuples with coordinates
    - how to: coordenadas=tuple(list_lat,list_ong)'''
    
    airports_ubi=[]
    for lat,long in zip(list_lat,list_long):
        ubication=(float(lat),float(long))
        airports_ubi.append(ubication)
    return airports_ubi

# - Geolocating with Folium

In [32]:
def map_adding(lista,nombre,colores):
    '''
    lista== column of the final dataset
    nombre== kind of facility
    colores == colour for this kind of facility
    '''
    for i in lista:
        folium.CircleMarker(
            location=[i[0],i[1]],
            radius=50,
            popup=nombre,
            color=colores,
            fill=True,
            fill_color=colores
        ).add_to(world_map)

# Creating coord tuples with my ubications
- it take the list of latitudes and list of longitudes for each ubication and turn it into a list of tuples (because is the designed mode to use haversine library)

In [6]:
airports_ubi=coord_tuple(lat_airport,lon_airport)

In [7]:
starbucks_ubi=coord_tuple(star_lat,star_long)

In [10]:
offices_ubi=coord_tuple(office_lat,office_long)

# Getting the closest distance from offices
- to all airports in my dataset (the most important in the world)
- I tried the scipy module for this purpose, but all i found it was uncompatible with gps coordinates, so, i downloaded the haversine library and worked with it

In [11]:
# using the function defined above
distance_airport_km, ubi_airport = closest_and_ubi(offices_ubi,airports_ubi)

In [12]:
distance_starbucks_km, ubi_starbucks=closest_and_ubi(offices_ubi,starbucks_ubi)

In [13]:
distance_kindergarten_km, ubi_kindergarten=closest_and_ubi(offices_ubi,kindergarten_ubi)

# Creating my dataframe with all the information:

In [18]:
data=pd.DataFrame({"offices":offices_ubi, 
                   "distance_mobility_point(km)":distance_airport_km,
                   "distance_starbucks(km)":distance_starbucks_km,
                   "distance_kindergartens(km)":distance_kindergarten_km,
                   "coords_airport":ubi_airport,
                   "coords_starbucks":ubi_starbucks,
                  "coords_kindergartens":ubi_kindergarten})


# Weight function to find the perfect spot
- I created a simple way to rank the preferred location
    - starbucks have great weight
    - it is not neccessary to have an airport very close, so airports weight reduce them in the function

In [21]:
data['Reverse_rank'] =(data["distance_mobility_point(km)"]*0.02+data["distance_starbucks(km)"]*100+data["distance_kindergartens(km)"]*40)

In [22]:
data=data.sort_values(by=["Reverse_rank"])

In [23]:
final_data=data.reset_index(drop=True)

In [24]:
final_data.head(5)

,offices,distance_mobility_point(km),distance_starbucks(km),distance_kindergartens(km),coords_airport,coords_starbucks,coords_kindergartens,Reverse_rank
0,"(40.739867, -73.990024)",1.2,0.0,0.6,"(40.7505, -73.9935)","(40.74, -73.99)","(40.74512371133894, -73.98815083551503)",24.024
1,"(40.72948, -74.000195)",1.8,0.1,0.4,"(40.714166999999996, -74.005833)","(40.73, -74.0)","(40.728274, -74.005063)",26.036
2,"(40.73906, -73.990911)",1.3,0.1,0.7,"(40.7505, -73.9935)","(40.74, -73.99)","(40.74512371133894, -73.98815083551503)",38.026
3,"(40.740512, -73.987794)",1.2,0.2,0.5,"(40.7505, -73.9935)","(40.74, -73.99)","(40.74512371133894, -73.98815083551503)",40.024
4,"(40.7438356, -73.9890149)",0.8,0.4,0.2,"(40.7505, -73.9935)","(40.74, -73.99)","(40.74512371133894, -73.98815083551503)",48.016


# Folium

In [25]:
# Creating name labels:


In [26]:
world_map=folium.Map(location=[final_data["offices"][0][0],final_data["offices"][0][1]])

In [19]:
#world_map

In [33]:
map_adding(final_data["offices"][0:5], "offices","red")

In [1]:
map_adding(final_data["coords_airport"][0:5],"public transport","grey")

NameError: name 'map_adding' is not defined

In [35]:
map_adding(final_data["coords_starbucks"][0:5],"starbucks","green")

In [36]:
map_adding(final_data["coords_kindergartens"][0:5], "kindergarten","blue")

In [37]:
world_map                 